## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coquimbo,CL,-29.9533,-71.3436,53.33,76,40,3.44,scattered clouds
1,1,Torbay,CA,47.6666,-52.7314,56.26,86,20,5.75,few clouds
2,2,Bintulu,MY,3.1667,113.0333,82.81,83,20,3.44,few clouds
3,3,Busselton,AU,-33.6500,115.3333,63.25,91,100,23.17,light rain
4,4,Punta Arenas,CL,-53.1500,-70.9167,33.91,93,100,6.91,fog


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bintulu,MY,3.1667,113.0333,82.81,83,20,3.44,few clouds
16,16,Hilo,US,19.7297,-155.0900,84.27,78,75,12.66,broken clouds
25,25,Poum,NC,-20.2333,164.0167,77.97,82,8,16.24,clear sky
26,26,Ixtapa,MX,20.7000,-105.2000,77.23,92,100,3.24,overcast clouds
29,29,Atuona,PF,-9.8000,-139.0333,78.12,78,35,16.40,scattered clouds
32,32,Laguna,US,38.4210,-121.4238,88.92,43,0,12.66,clear sky
33,33,Faanui,PF,-16.4833,-151.7500,78.58,69,0,8.95,clear sky
38,38,Puerto Cabezas,NI,14.0351,-83.3888,78.55,88,96,9.93,overcast clouds
42,42,Kimbe,PG,-5.5502,150.1429,89.10,56,93,6.67,overcast clouds
45,45,Sakakah,SA,29.9697,40.2064,78.37,15,8,6.17,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bintulu,MY,82.81,few clouds,3.1667,113.0333,
16,Hilo,US,84.27,broken clouds,19.7297,-155.0900,
25,Poum,NC,77.97,clear sky,-20.2333,164.0167,
26,Ixtapa,MX,77.23,overcast clouds,20.7000,-105.2000,
29,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,
32,Laguna,US,88.92,clear sky,38.4210,-121.4238,
33,Faanui,PF,78.58,clear sky,-16.4833,-151.7500,
38,Puerto Cabezas,NI,78.55,overcast clouds,14.0351,-83.3888,
42,Kimbe,PG,89.10,overcast clouds,-5.5502,150.1429,
45,Sakakah,SA,78.37,clear sky,29.9697,40.2064,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url =  "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [23]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bintulu,MY,82.81,few clouds,3.1667,113.0333,900 Boutique INN
16,Hilo,US,84.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
25,Poum,NC,77.97,clear sky,-20.2333,164.0167,
26,Ixtapa,MX,77.23,overcast clouds,20.7000,-105.2000,hotel la terminal
29,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,Villa Enata
32,Laguna,US,88.92,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
33,Faanui,PF,78.58,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
38,Puerto Cabezas,NI,78.55,overcast clouds,14.0351,-83.3888,Cabaña Caribe
42,Kimbe,PG,89.10,overcast clouds,-5.5502,150.1429,Kimbe Bay Hotel
45,Sakakah,SA,78.37,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bintulu,MY,82.81,few clouds,3.1667,113.0333,900 Boutique INN
16,Hilo,US,84.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
26,Ixtapa,MX,77.23,overcast clouds,20.7000,-105.2000,hotel la terminal
29,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,Villa Enata
32,Laguna,US,88.92,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
33,Faanui,PF,78.58,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
38,Puerto Cabezas,NI,78.55,overcast clouds,14.0351,-83.3888,Cabaña Caribe
42,Kimbe,PG,89.10,overcast clouds,-5.5502,150.1429,Kimbe Bay Hotel
45,Sakakah,SA,78.37,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
47,Novobureyskiy,RU,78.21,overcast clouds,49.7980,129.8774,-


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))